In [20]:
import os
os.chdir("F:/63Moons/entree_data/entree/data")
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("chicago.csv")
df.head()

,Restaurant_id,Restaurant_name,Review_feature,feature_name1,feature_name2,feature_name3,feature_name4,feature_name5,feature_name6,feature_name7,...,feature_name15,feature_name16,feature_name17,feature_name18,feature_name19,feature_name20,feature_name21,feature_name22,feature_name23,feature_name24
0,0,Moti Mahal,Short Drive Carry in Wine and Beer No Reservat...,Short Drive,Carry in Wine and Beer,No Reservations,Buffet Dining,Indian,Excellent Food,Good Service,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Village,Cab Weekend Brunch Parking/Valet After Hours D...,Cab,Weekend Brunch,Parking/Valet,After Hours Dining,Late Night Menu,Weekend Brunch,Relaxed Senior Scene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Millrose Brewing Company,Long Drive Weekend Brunch Pub Feel Singles Sce...,Long Drive,Weekend Brunch,Pub Feel,Singles Scene,See the Game,Parking/Valet,Weekend Brunch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Dover Straits,Long Drive Pre-theater Dining Parking/Valet We...,Long Drive,Pre-theater Dining,Parking/Valet,Weekend Brunch,Seafood,Excellent Food,Excellent Service,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Eat Your Hearts Out,Short Drive Weekend Brunch Weekend Brunch Quir...,Short Drive,Weekend Brunch,Weekend Brunch,Quirky,Health Conscious Menus,Creative,Creative,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.iloc[:,:3]
df

,Restaurant_id,Restaurant_name,Review_feature
0,0,Moti Mahal,Short Drive Carry in Wine and Beer No Reservat...
1,1,Village,Cab Weekend Brunch Parking/Valet After Hours D...
2,2,Millrose Brewing Company,Long Drive Weekend Brunch Pub Feel Singles Sce...
3,3,Dover Straits,Long Drive Pre-theater Dining Parking/Valet We...
4,4,Eat Your Hearts Out,Short Drive Weekend Brunch Weekend Brunch Quir...
5,5,Pizzeria Uno & Due,Cab Weekend Brunch After Hours Dining Late Nig...
6,6,Trattoria Franco,Short Drive Little Known But Well Liked Italia...
7,7,Little Bucharest,Short Drive After Hours Dining Late Night Menu...
8,8,Pattaya,Cab Thai Good Food Excellent Service Fair Deco...
9,9,House of Hunan,Cab Private Parties Private Rooms Available Bu...


In [4]:
df.set_index("Restaurant_id")

,Restaurant_name,Review_feature
Restaurant_id,,
0,Moti Mahal,Short Drive Carry in Wine and Beer No Reservat...
1,Village,Cab Weekend Brunch Parking/Valet After Hours D...
2,Millrose Brewing Company,Long Drive Weekend Brunch Pub Feel Singles Sce...
3,Dover Straits,Long Drive Pre-theater Dining Parking/Valet We...
4,Eat Your Hearts Out,Short Drive Weekend Brunch Weekend Brunch Quir...
5,Pizzeria Uno & Due,Cab Weekend Brunch After Hours Dining Late Nig...
6,Trattoria Franco,Short Drive Little Known But Well Liked Italia...
7,Little Bucharest,Short Drive After Hours Dining Late Night Menu...
8,Pattaya,Cab Thai Good Food Excellent Service Fair Deco...


In [89]:
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

from keras.models import Model
from keras.layers import Embedding, Input, Reshape
from keras.layers.merge import Dot

In [6]:
from nltk.corpus import stopwords
from string import punctuation
stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
Lemmatizer = WordNetLemmatizer()
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\sanat\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
def my_tokenizer(s):
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t)>2] #remove words lesser than 2 in length
    tokens = [Lemmatizer.lemmatize(t) for t in tokens] #lemmatize words
    tokens = [t for t in tokens if t not in stuff_to_be_removed] #remove stopwords and punctuation
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)] # remove digits
    return tokens

In [9]:
import pandas as pd
text = df["Review_feature"].astype(str).tolist()[:100]

In [10]:
text

['Short Drive Carry in Wine and Beer No Reservations Buffet Dining Indian Excellent Food Good Service Fair Decor below $15               ',
 'Cab Weekend Brunch Parking/Valet After Hours Dining Late Night Menu Weekend Brunch Relaxed Senior Scene Private Parties Private Rooms Available Italian Excellent Food Excellent Service Extraordinary Decor $15-$30          ',
 'Long Drive Weekend Brunch Pub Feel Singles Scene See the Game Parking/Valet Weekend Brunch Private Parties Private Rooms Available American (Traditional) Excellent Food Excellent Service Extraordinary Decor $15-$30          ',
 'Long Drive Pre-theater Dining Parking/Valet Weekend Brunch Seafood Excellent Food Excellent Service Excellent Decor $15-$30               ',
 'Short Drive Weekend Brunch Weekend Brunch Quirky Health Conscious Menus Creative Creative Health Food Excellent Food Excellent Service Excellent Decor $15-$30            ',
 'Cab Weekend Brunch After Hours Dining Late Night Menu Good for Younger Kids For the 

In [11]:
text = [my_tokenizer(s) for s in text]

In [12]:
from gensim.models import Word2Vec

In [13]:
model = Word2Vec(text)

C:\Users\sanat\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [14]:
model

In [15]:
#concordance and similar words
model.wv.most_similar("pizza")

[('business', 0.5598604083061218),
 ('brunch', 0.5322573184967041),
 ('food', 0.5313950777053833),
 ('good', 0.5235936641693115),
 ('excellent', 0.5138721466064453),
 ('american', 0.5114978551864624),
 ('weekend', 0.5107264518737793),
 ('hour', 0.5007601976394653),
 ('wine', 0.5006881952285767),
 ('liked', 0.4998306632041931)]

In [16]:
#how are they similar
model.wv.similarity("good", 'great')  

0.57809716

In [17]:
from sklearn.decomposition import PCA
from matplotlib import pyplot


X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)


# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

C:\Users\sanat\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


<Figure size 640x480 with 1 Axes>

In [26]:
import random
positive = random.sample(text, 60)
negative  = random.sample(model.wv.vocab.keys(), 70)

In [ ]:
# countries =[{'cc': 'ca', 'cc3': 'can', 'name': 'Canada'},
#  {'cc': 'tm', 'cc3': 'tkm', 'name': 'Turkmenistan'},
#  {'cc': 'et', 'cc3': 'eth', 'name': 'Ethiopia'},
#  {'cc': 'sz', 'cc3': 'swz', 'name': 'Swaziland'},
#  {'cc': 'cz', 'cc3': 'cze', 'name': 'Czech_Republic'},
#  {'cc': 'cm', 'cc3': 'cmr', 'name': 'Cameroon'},
#  {'cc': 'ae', 'cc3': 'are', 'name': 'UAE'},
#  {'cc': 'lr', 'cc3': 'lbr', 'name': 'Liberia'},
#  {'cc': 'nl', 'cc3': 'nld', 'name': 'Netherlands'},
#  {'cc': 'tl', 'cc3': 'tls', 'name': 'East_Timor'}]
# [x['name'] for x in random.sample(countries, 4)]

In [66]:

labelled = [(p, 1) for p in positive] + [(n, 0) for n in negative]
random.shuffle(labelled)

X = np.asarray([w for w, l in labelled])
y = np.asarray([l for w, l in labelled])
X.shape, y.shape


((130,), (130,))

In [83]:
q=([w for w, l in labelled])
X = pd.DataFrame(np.asarray(q))

y = pd.DataFrame(np.asarray([l for w, l in labelled]))
y.shape

(130, 1)

In [108]:

def food_embedding_model(embedding_size=50):
    link = Input(X)
    movie = Input(y)
    review_embedding = Embedding(name='review_embedding', 
                               input_dim=30, 
                               output_dim=30)(link)
    
    
    restaurant_embedding = Embedding(name='restaurant_embedding', 
                                input_dim=30, 
                                output_dim=30)(movie)
    
    
    dot = Dot(name='dot_product', normalize=True, axes=2)([review_embedding, restaurant_embedding])
    
    
    #merged = Reshape((1,1))(dot)
    
    model = Model(inputs=[link, movie], outputs=[dot])
    model.compile(optimizer='nadam', loss='mse')
    return model

model = food_embedding_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_46 (InputLayer)           (None, 0)            0                                            
__________________________________________________________________________________________________
input_47 (InputLayer)           (None, 0)            0                                            
__________________________________________________________________________________________________
review_embedding (Embedding)    (None, 0, 30)        900         input_46[0][0]                   
__________________________________________________________________________________________________
restaurant_embedding (Embedding (None, 0, 30)        900         input_47[0][0]                   
__________________________________________________________________________________________________
dot_produc

In [110]:
random.seed(5)

def batchifier(pairs, positive_samples=50, negative_ratio=10):
    batch_size = positive_samples * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    while True:
        for idx, (link_id, movie_id) in enumerate(random.sample(pairs, positive_samples)):
            batch[idx, :] = (link_id, movie_id, 1)
        idx = positive_samples
        while idx < batch_size:
            movie_id = random.randrange(len(movie_to_idx))
            link_id = random.randrange(len(top_links))
            if not (link_id, movie_id) in pairs_set:
                batch[idx, :] = (link_id, movie_id, -1)
                idx += 1
        np.random.shuffle(batch)
        yield {'link': batch[:, 0], 'movie': batch[:, 1]}, batch[:, 2]

#next(batchifier(pairs, positive_samples=3, negative_ratio=2))

In [111]:
# TRAINING_FRACTION = 0.3
# cut_off = int(TRAINING_FRACTION * len(labelled))
# clf = svm.SVC(kernel='linear')
# clf.fit(X[:cut_off], y[:cut_off])

In [112]:
positive_samples_per_batch = 512

model.fit_generator(
    batchifier(pairs, positive_samples=positive_samples_per_batch, negative_ratio=10),
    epochs=15,
    steps_per_epoch=len(pairs) // positive_samples_per_batch,
    verbose=2
)

NameError: name 'pairs' is not defined